In [ ]:
'''Imports'''
import matplotlib.pyplot as plt
import cv2
import numpy as np
from mpl_toolkits.axes_grid1 import ImageGrid

In [ ]:
'''Define helpers'''

def find_connected_components(image):

    # Binarize image and make a negative (cv2 connected components works better with a black background,
    # and the background of the maps are white when using the function in this implementation)
    bin_img = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)[1]
    bin_img = 255 - bin_img

    # Find connected components
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(bin_img, connectivity = 4)

    #Map component labels to hue val
    label_hue = np.uint8(179*labels/np.max(labels))
    blank_ch = 255*np.ones_like(label_hue)
    labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])

    # convert to BGR for display
    labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)

    # Return labeled image and other params
    return labeled_img, labels, num_labels, stats


def remove_small_connected_components(image, labels, num_labels, stats, min_size = 20):
    
    # Create an output image with the same dimensions as the input
    output_img = np.zeros_like(image)
    
    # Iterate through connected components
    for label in range(1, num_labels):
        if stats[label, cv2.CC_STAT_AREA] >= min_size:
            output_img[labels == label] = 255
    
    # Convert to gray and negative (make the background white again since the
    # find_connected_components function made it black)
    output_img = cv2.cvtColor(output_img, cv2.COLOR_BGR2GRAY)
    output_img = np.uint8(255 - output_img)

    return output_img

In [ ]:
'''Clean the maps'''

# Original image
map = plt.imread('belinson_martin_notags.pgm')

# Extraction of gray levels assuming the map only contains 3 gray levels
gray_levels = np.unique(map)
black, gray, white = gray_levels[0], gray_levels[1], gray_levels[2]

# Median filtered
median = cv2.medianBlur(map, 5)

# Dilation
dilation_kernel = np.ones((3,3),np.uint8)
dilation = cv2.dilate(median, dilation_kernel, iterations = 1)

# Remove small connected components
labeled_img, labels, num_labels, stats = find_connected_components(dilation)
no_small_components = remove_small_connected_components(labeled_img, labels, num_labels, stats)
no_small_components[dilation == gray] = gray

# Edge detection + preparing to merge with non-binarized image
edges = cv2.Canny(image = no_small_components, threshold1 = 100, threshold2 = 200)
edges += 1

# Merge edges with non-binarized image
final_not_eroded = no_small_components * edges

# Erosion
erosion_kernel = np.ones((1,3), np.uint8)
final_eroded = cv2.erode(final_not_eroded, erosion_kernel, iterations = 1)

In [ ]:
'''Display results'''

fig = plt.figure(figsize = (15,10))
grid = ImageGrid(fig, 111, nrows_ncols = (1,7), axes_pad = 0.4)

grid[0].imshow(map, cmap = 'gray')
grid[0].set_title('Original Image', fontsize = 15)
grid[0].axis('off')

grid[1].imshow(median, cmap = 'gray')
grid[1].set_title('Median Filtered', fontsize = 15)
grid[1].axis('off')

grid[2].imshow(dilation, cmap = 'gray')
grid[2].set_title('Dilated Image', fontsize = 15)
grid[2].axis('off')

grid[3].imshow(no_small_components, cmap = 'gray')
grid[3].set_title('No\nSmall Components', fontsize = 15)
grid[3].axis('off')

grid[4].imshow(edges, cmap = 'gray')
grid[4].set_title('Edges', fontsize = 15)
grid[4].axis('off')

grid[5].imshow(final_not_eroded, cmap = 'gray')
grid[5].set_title('Final Map\nNot Eroded', fontsize = 15)
grid[5].axis('off')

grid[6].imshow(final_eroded, cmap = 'gray')
grid[6].set_title('Final Map\nEroded', fontsize = 15)
grid[6].axis('off')

In [ ]:
BUFFER_IDS = {'buffer_6bd928a55b4ab48c8f6b3af358d3f58c1aeee409': 'dat:tts_audio/VOICE_EXPLAIN_FINGER_OR_WAND_HEBREW.mp3', 'buffer_b943affcd0e165eaac6192c0c87f52fbeb2672f1': 'dat:tts_audio/VOICE_DANIEL_IS_A_MEME_HEBREW.mp3', 'buffer_40c177a0006cbbba0b4b5e95ec8f449f6348bd08': 'dat:tts_audio/VOICE_TRIVIA_QUESTION_1_MEDIUM_HEBREW.mp3', 'buffer_f9aa1dd57aa7c1885319ef4c1d28d45aecb43925': 'dat:tts_audio/VOICE_CARD_MATCH_4_HEBREW.mp3', 'buffer_3beff3faa17caeda63839b7fd33b9bf29a2c4dda': 'dat:tts_audio/VOICE_STOP_BUT_REGRET_HEBREW.mp3', 'buffer_5e3b5f74a8d8586941817759b05afec10141b04b': 'dat:tts_audio/VOICE_STRAIGHT_HEAD_HEBREW.mp3', 'buffer_2488a3a7dfaae91ec228b4eba370ddc2f79c32ac': 'dat:tts_audio/VOICE_CONTINUE_STAGE_HEBREW.mp3', 'buffer_eaa5b087aa37653bcb8fcd72e51e19b4d5d9eae4': 'dat:tts_audio/VOICE_CARD_MISMATCH_1_HEBREW.mp3', 'buffer_19e6e4f4c9734532c89d80886dc90f152a20a6b1': 'dat:tts_audio/VOICE_DIFFERENCE_GAME_HEBREW.mp3', 'buffer_c2fa1b4c0f5dc419d9911c49577712423c847309': 'dat:tts_audio/VOICE_END_TREATMENT_HEBREW.mp3', 'buffer_2915cbd1e4d2a3668bf36d072317776fd954c74d': 'dat:tts_audio/VOICE_CARD_MISMATCH_2_HEBREW.mp3', 'buffer_a46f436a7a5c9e68bb9b9875382707b39a872b76': 'dat:tts_audio/VOICE_INTRODUCTION_HEBREW.mp3', 'buffer_71ee06245388d55fb12cf8c21e226382c520d6c7': 'dat:tts_audio/VOICE_PLEASE_MOVE_HEBREW.mp3', 'buffer_3f0afc4ca95d20c7c0168b29395230a5f6c9a116': 'dat:tts_audio/VOICE_NO_TIME_FOR_CAUTION_HEBREW.mp3', 'buffer_9f457abe8d3dbb6e950c3ee8da72d82a554db952': 'dat:tts_audio/VOICE_STOP_CONDITION_HEBREW.mp3', 'buffer_fe3489c3eea2cd215e5a12629a80cfe71430d080': 'dat:tts_audio/VOICE_MOVE_UNTIL_STOP_HEBREW.mp3', 'buffer_ee35c25f1f02e59976d89564607e566707661e95': 'dat:tts_audio/VOICE_BELINSON_HEBREW.mp3', 'buffer_2f34f89aa3c22c17a452f0fa1cf49874e2a07cd4': 'dat:tts_audio/VOICE_FEEDBACK_OKAY_HEBREW.mp3', 'buffer_29cb59cb20375320d214de84de48590123c0156c': 'dat:tts_audio/VOICE_FINGER_OR_WAND_CHOICE_HEBREW.mp3', 'buffer_f4b4f17485acfab52cc1b0091b4b87127ee12d36': 'dat:tts_audio/VOICE_TRIVIA_QUESTION_2_MEDIUM_HEBREW.mp3', 'buffer_0d5edf264a07689eddac7b5d88798a29a2184b16': 'dat:tts_audio/VOICE_SIT_STRAIGHT_HEBREW.mp3', 'buffer_b8877992189b3e1fcd6fa1bdf9b7336e24ccd687': 'dat:tts_audio/VOICE_COULDNT_REACH_DESTINATION_HEBREW.mp3', 'buffer_99c566ab6c7b27675f2bc84f29c494c04b5734a6': 'dat:tts_audio/VOICE_TOUCHSCREEN_HEBREW.mp3', 'buffer_2ea850178e961d2dc7bc8566c115bf415d18b55e': 'dat:tts_audio/VOICE_CARD_MATCH_LAST_HEBREW.mp3', 'buffer_80e32a6fdd1f33e717ee162a51b20199e4f642f5': 'dat:tts_audio/VOICE_PRESS_BUTTON_HEBREW.mp3', 'buffer_0d0a7bfb8c9661a810fcd0846b48c0592867fe5f': 'dat:tts_audio/VOICE_GERIATRIC_HEBREW.mp3', 'buffer_ee2bcc2c24d5f0244e162938aa853549e932d346': 'dat:tts_audio/VOICE_GET_OUT_OF_THE_WAY_HEBREW.mp3', 'buffer_94fc3ba5ede4c2fd82e7a33fd04103113e4b884d': 'dat:tts_audio/VOICE_BACKUP_CURRENT_TIME_HEBREW.mp3', 'buffer_8bf82bf7c402cbd6400e531a6ccb44eb13c802a4': 'dat:tts_audio/VOICE_ARRIVING_HEBREW.mp3', 'buffer_5ce9d45128516c825688fa7eddeb75599ca2bfa8': 'dat:tts_audio/VOICE_TRIVIA_QUESTION_3_MEDIUM_HEBREW.mp3', 'buffer_3bb95fea9f8f96a24f887bbb52725e0ec2e53a47': 'dat:tts_audio/VOICE_GUIDELINES_HEBREW.mp3', 'buffer_2c29bcc536ad004b896fcfb4e0cbb32e234e964d': 'dat:tts_audio/VOICE_ABORTED_BY_STAFF_HEBREW.mp3', 'buffer_b0f70c28d3e42c57f3242102c971362af173b6c4': 'dat:tts_audio/VOICE_TAKE_STICK_HEBREW.mp3', 'buffer_4956b3ac1ac7f68ac736b4bee2a86a1f04a7295b': 'dat:tts_audio/VOICE_EXPLAIN_SESSION_HEBREW.mp3', 'buffer_f8cbfb5c515ea454858e83cf93e99791a4598f84': 'dat:tts_audio/VOICE_CARD_MATCH_1_HEBREW.mp3', 'buffer_032b4532d372a8ccad188e2dc1da37ec4f89536c': 'dat:tts_audio/VOICE_CLEAR_APPROACH_OBSTACLES_HEBREW.mp3', 'buffer_6ab9000f33c9e8f8f50884ad713dff96a981c906': 'dat:tts_audio/VOICE_MEMORY_GAME_HEBREW.mp3', 'buffer_a75e9ad211b2c706fe802e0a0b2b692433ca4a4d': 'dat:tts_audio/wrong_answer_sound.mp3', 'buffer_66533affed04e110bc473e9c1adcec419e9efe9f': 'dat:tts_audio/VOICE_COULDNT_GET_CLOSE_ENOUGH_HEBREW.mp3', 'buffer_2db999f119ba3acd84f0a8a4288c5b566322c0e7': 'dat:tts_audio/VOICE_PLEASE_TUCK_LEGS_HEBREW.mp3', 'buffer_b7886449632bc78b40869ef3cacf175a7c3fce5a': 'dat:tts_audio/gary_response.mp3', 'buffer_9be7d4a03d4968f76e676c2fd7ff89af745d2c12': 'dat:tts_audio/VOICE_CARD_MATCH_3_HEBREW.mp3', 'buffer_b05a749ff1d050be12f8213d5f3f74f30bf43953': 'dat:tts_audio/VOICE_TEN_SECONDS_REMAINING_HEBREW.mp3', 'buffer_d86d1f9d6cedf74bffd32e9b7c5f811a2debb57b': 'dat:tts_audio/VOICE_ABORTED_BY_PATIENT_HEBREW.mp3', 'buffer_4d46d04165c1a4b6fc0bd196f8c2adaf68db4218': 'dat:tts_audio/VOICE_BEFORE_ACTIVITIES_HEBREW.mp3', 'buffer_f66ad62362359271cb5a5171ac97caa133cb1374': 'dat:tts_audio/VOICE_MOVING_BACKWARDS_HEBREW.mp3', 'buffer_2b417b476572cb11a6d5eb308c62c53ec216866c': 'dat:tts_audio/VOICE_APPROACHING_HEBREW.mp3', 'buffer_6e50dada06e53d75f27f876b41e8b036d838bb36': 'dat:tts_audio/VOICE_RETURN_STICK_HEBREW.mp3', 'buffer_b9cd334216fb9d505277a499572a867d3e3f0c5f': 'dat:tts_audio/VOICE_LEGS_ON_FLOOR_HEBREW.mp3', 'buffer_9cb38e65434607304cfc5fef82fffe39033649b1': 'dat:tts_audio/VOICE_ABORT_REASON_HEBREW.mp3', 'buffer_93d8357947c7a4217f04c604ce3a461074f80c8f': 'dat:tts_audio/VOICE_ACTIVITIES_HEBREW.mp3', 'buffer_27baa41615db89066598fcf2a5b2455431d39618': 'dat:tts_audio/VOICE_BAD_ID_HEBREW.mp3', 'buffer_c06f70a57d1e52fb9dae29add80f8b21cf41a7f7': 'dat:tts_audio/VOICE_ASKING_FLEET_FOR_HELP_HEBREW.mp3', 'buffer_eb7b91eb6283050ceda12994d56b3bb114ecdba3': 'dat:tts_audio/button_pressed_sound.wav', 'buffer_64e1d713891611130931a14aa1172b8779efcb7d': 'dat:tts_audio/VOICE_MINUTE_REMAINING_HEBREW.mp3', 'buffer_fd6cb578ccf40d98a12aa760906b0a3e79f3b4cf': 'dat:tts_audio/VOICE_AFTER_FEEDBACK_HEBREW.mp3', 'buffer_ed5cf3192c7c64e9f47692b7b9f486a110743f8c': 'dat:tts_audio/VOICE_FEEDBACK_GOOD_HEBREW.mp3', 'buffer_9d01eaf881412199bc181654797926b74eaa9e17': 'dat:tts_audio/VOICE_TRIVIA_GAME_HEBREW.mp3', 'buffer_7b9a8ffe29b8317a49036adfe080bccfcaa2d70e': 'dat:tts_audio/VOICE_FEEDBACK_BAD_HEBREW.mp3', 'buffer_a5deafbfbd81b17444fca32506ad12ee86649057': 'dat:tts_audio/VOICE_GAME_COMPLETED_HEBREW.mp3', 'buffer_3a28ce7918a646f6827092a68bd8df081f7b32e3': 'dat:tts_audio/VOICE_SESSION_EMPHASES_HEBREW.mp3', 'buffer_2ec3a5d658df61154ffa1143cb9f6098f8b1bb37': 'dat:tts_audio/VOICE_VERIFY_PATIENT_HEBREW.mp3', 'buffer_fd365598a9433eba6947ab598fd52834fb40f47c': 'dat:tts_audio/VOICE_CARD_MISMATCH_N_HEBREW.mp3', 'buffer_438f7b223662921b839e8e8bc9838d31ae3c6dc3': 'dat:tts_audio/VOICE_CARD_MATCH_2_HEBREW.mp3', 'buffer_d8cf99b1b53e0872d3ec83ed12fd35dc6d77c532': 'dat:tts_audio/VOICE_BEFORE_COGNISHINE_HEBREW.mp3', 'buffer_383946a5b316acb9f3e3e2bf1c3ef1efc181656b': 'dat:tts_audio/VOICE_MOVING_FORWARDS_HEBREW.mp3', 'buffer_589feb3ea85c5b18c79e76cd523b7ce9cc875b96': 'dat:tts_audio/VOICE_FEEDBACK_HEBREW.mp3'}
REC_TIMES = {'VOICE_EXPLAIN_FINGER_OR_WAND_HEBREW.mp3': 7.6180479526519775, 'VOICE_DANIEL_IS_A_MEME_HEBREW.mp3': 6.820580005645752, 'VOICE_TRIVIA_QUESTION_1_MEDIUM_HEBREW.mp3': 2.9781696796417236, 'VOICE_CARD_MATCH_4_HEBREW.mp3': 2.7024097442626953, 'VOICE_STOP_BUT_REGRET_HEBREW.mp3': 3.41317081451416, 'VOICE_STRAIGHT_HEAD_HEBREW.mp3': 3.85896372795105, 'VOICE_CONTINUE_STAGE_HEBREW.mp3': 4.171376943588257, 'VOICE_CARD_MISMATCH_1_HEBREW.mp3': 2.0405919551849365, 'VOICE_DIFFERENCE_GAME_HEBREW.mp3': 11.82339596748352, 'VOICE_END_TREATMENT_HEBREW.mp3': 7.091966152191162, 'VOICE_CARD_MISMATCH_2_HEBREW.mp3': 1.9844322204589844, 'VOICE_INTRODUCTION_HEBREW.mp3': 15.456987857818604, 'VOICE_PLEASE_MOVE_HEBREW.mp3': 3.005232810974121, 'VOICE_NO_TIME_FOR_CAUTION_HEBREW.mp3': 12.482739686965942, 'VOICE_STOP_CONDITION_HEBREW.mp3': 6.905527353286743, 'VOICE_MOVE_UNTIL_STOP_HEBREW.mp3': 7.6995673179626465, 'VOICE_BELINSON_HEBREW.mp3': 7.959827184677124, 'VOICE_FEEDBACK_OKAY_HEBREW.mp3': 4.759559392929077, 'VOICE_FINGER_OR_WAND_CHOICE_HEBREW.mp3': 8.795055150985718, 'VOICE_TRIVIA_QUESTION_2_MEDIUM_HEBREW.mp3': 4.114856719970703, 'VOICE_SIT_STRAIGHT_HEBREW.mp3': 7.865539073944092, 'VOICE_COULDNT_REACH_DESTINATION_HEBREW.mp3': 4.136565208435059, 'VOICE_TOUCHSCREEN_HEBREW.mp3': 7.542022466659546, 'VOICE_CARD_MATCH_LAST_HEBREW.mp3': 3.545989990234375, 'VOICE_PRESS_BUTTON_HEBREW.mp3': 3.4230504035949707, 'VOICE_GERIATRIC_HEBREW.mp3': 4.2677247524261475, 'VOICE_GET_OUT_OF_THE_WAY_HEBREW.mp3': 3.442479133605957, 'VOICE_BACKUP_CURRENT_TIME_HEBREW.mp3': 9.06003713607788, 'VOICE_ARRIVING_HEBREW.mp3': 7.096929311752319, 'VOICE_TRIVIA_QUESTION_3_MEDIUM_HEBREW.mp3': 5.977142333984375, 'VOICE_GUIDELINES_HEBREW.mp3': 7.888633489608765, 'VOICE_ABORTED_BY_STAFF_HEBREW.mp3': 7.41620397567749, 'VOICE_TAKE_STICK_HEBREW.mp3': 15.97906494140625, 'VOICE_EXPLAIN_SESSION_HEBREW.mp3': 4.185892581939697, 'VOICE_CARD_MATCH_1_HEBREW.mp3': 2.721837043762207, 'VOICE_CLEAR_APPROACH_OBSTACLES_HEBREW.mp3': 4.213702201843262, 'VOICE_MEMORY_GAME_HEBREW.mp3': 18.324275255203247, 'wrong_answer_sound.mp3': 2.797577381134033, 'VOICE_COULDNT_GET_CLOSE_ENOUGH_HEBREW.mp3': 5.214289426803589, 'VOICE_PLEASE_TUCK_LEGS_HEBREW.mp3': 6.682427644729614, 'gary_response.mp3': 8.8454008102417, 'VOICE_CARD_MATCH_3_HEBREW.mp3': 1.7325165271759033, 'VOICE_TEN_SECONDS_REMAINING_HEBREW.mp3': 3.9221034049987793, 'VOICE_ABORTED_BY_PATIENT_HEBREW.mp3': 4.079399585723877, 'VOICE_BEFORE_ACTIVITIES_HEBREW.mp3': 14.271104097366333, 'VOICE_MOVING_BACKWARDS_HEBREW.mp3': 4.315164804458618, 'VOICE_APPROACHING_HEBREW.mp3': 4.3822855949401855, 'VOICE_RETURN_STICK_HEBREW.mp3': 6.1239800453186035, 'VOICE_LEGS_ON_FLOOR_HEBREW.mp3': 3.53191876411438, 'VOICE_ABORT_REASON_HEBREW.mp3': 4.072263479232788, 'VOICE_ACTIVITIES_HEBREW.mp3': 8.086297273635864, 'VOICE_BAD_ID_HEBREW.mp3': 4.776260137557983, 'VOICE_ASKING_FLEET_FOR_HELP_HEBREW.mp3': 4.1013829708099365, 'button_pressed_sound.wav': 2.8774352073669434, 'VOICE_MINUTE_REMAINING_HEBREW.mp3': 6.745274305343628, 'VOICE_AFTER_FEEDBACK_HEBREW.mp3': 3.478609800338745, 'VOICE_FEEDBACK_GOOD_HEBREW.mp3': 3.8691985607147217, 'VOICE_TRIVIA_GAME_HEBREW.mp3': 8.492942333221436, 'VOICE_FEEDBACK_BAD_HEBREW.mp3': 6.340404748916626, 'VOICE_GAME_COMPLETED_HEBREW.mp3': 4.082958459854126, 'VOICE_SESSION_EMPHASES_HEBREW.mp3': 4.129499912261963, 'VOICE_VERIFY_PATIENT_HEBREW.mp3': 6.386006832122803, 'VOICE_CARD_MISMATCH_N_HEBREW.mp3': 1.9280297756195068, 'VOICE_CARD_MATCH_2_HEBREW.mp3': 2.7187535762786865, 'VOICE_BEFORE_COGNISHINE_HEBREW.mp3': 7.381462812423706, 'VOICE_MOVING_FORWARDS_HEBREW.mp3': 6.97395396232605, 'VOICE_FEEDBACK_HEBREW.mp3': 5.14737606048584}

In [ ]:
def combine_dicts(dict_ids, dict_timestamps):
        combined_dict = {}
        for name in dict_ids:
            combined_dict[name] = {
                'buffer_id': dict_ids[name],
                'timestamp': dict_timestamps[name]
            }

        return combined_dict

def reverse_dict(original_dict):
        reversed_dict = {value: key for key, value in original_dict.items()}
        return reversed_dict

def strip_prefix_suffix_from_keys(orig_dict, prefix):
        prefix_len = len(prefix)
        # Create a new dictionary with stripped keys
        new_dict = {}
        for key, value in orig_dict.items():
            if key.startswith(prefix):
                key = key[prefix_len:]
            if key.endswith(prefix):
                key = key[:-prefix_len]
            new_dict[key] = value
        return new_dict
    

In [ ]:
buffer_dict = reverse_dict(BUFFER_IDS)
buffer_dict = strip_prefix_suffix_from_keys(orig_dict = buffer_dict,
                                                prefix = 'dat:tts_audio/')
buffer_dict = strip_prefix_suffix_from_keys(orig_dict = buffer_dict,
                                                    prefix = '.mp3')

print(f'buffer dict: {buffer_dict}')

In [1]:
import os
from google.cloud import texttospeech
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'client_service_key_3.json'
text_to_speech_client = texttospeech.TextToSpeechClient()


In [2]:
def text_to_speech(text,
                    language='en-GB',
                    name='en-GB-Neural2-B',
                    audio_type='mp3',
                    leds=True):
        """
        INPUTS:
                text - A text for the robot to speak

        OUTPUTS:
                This function doesn't return any outputs, it speaks 'text'
        """

        synthesized_input = texttospeech.SynthesisInput(text=text)
        voice = texttospeech.VoiceSelectionParams(
            language_code=language,
            name=name,
            ssml_gender=texttospeech.SsmlVoiceGender.MALE)
        audio_config = texttospeech.AudioConfig(
                                audio_encoding=texttospeech.AudioEncoding.MP3,
                                speaking_rate = 0.9
                                )
        response = text_to_speech_client.synthesize_speech(
                                                    input=synthesized_input,
                                                    voice=voice,
                                                    audio_config=audio_config
                                                    )
        
        print(f'response: {response}')

        with open(f'gary_response.{audio_type}', 'wb') as gary_response:
            gary_response.write(response.audio_content)


In [3]:
text_to_speech(text = 'hello world')

response: audio_content: "\377\363\204\304\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\377\363\204\304\000(\031q\314\000\313\014\225=\032N\225\326\225h>_\362\323\227,\274h\000H\205\210\327\035\310\301 H\022\000\370&\004\300\230\023\003\340\334G30Xpx`$\t\002@\220#\211ffk\327\276\275y\301\201\